In [1]:
%matplotlib inline

In [2]:
import sys
sys.path += [".."]
import yamath.newton as newton

In [3]:
import logging

In [4]:
from scipy.integrate import ode

In [5]:
def Lorenz(t, vy, p, r, b):
    x, y, z = vy
    return [p*(y-x), x*(r-z)-y, x*y-b*z]

In [6]:
r = ode(Lorenz)
r.set_integrator("dopri5")
r.set_f_params(10., 50., 8./3.)
r.set_initial_value((1., 0., 0.), 0)

In [7]:
z_ave = 45.276968019646574
dt = 1e-3

In [8]:
def Poincare(y, count=1):
    """Poincare map
    """
    tl = []
    c = 0
    r.set_initial_value(y, 0)
    y_pre = r.y.copy()
    step = 0
    while True:
        r.integrate(r.t + dt)
        step += 1
        if not r.successful():
            raise RuntimeError("Iteration failed")
        tl.append([r.t,] + list(r.y))
        if r.t > dt and y_pre[2] < z_ave and r.y[2] > z_ave:
            c += 1
            if c >= count:
                a = z_ave - y_pre[2]
                b = r.y[2] - z_ave
                y_p = (b * y_pre + a * r.y) / (a + b) # 線形に補間
                # print(step)
                return y_p, pd.DataFrame(tl, columns=["t", "x", "y", "z"]).set_index("t")
        y_pre = r.y.copy()

In [9]:
def P(x):
    """ Poincare Map (on Poincare section)
    """
    yp, _ = Poincare((x[0], x[1], z_ave))
    Py = yp[0:2] - x
    return Py

In [10]:
y0 = np.array([-13.03147007,  -0.52028421,  61.0841961 ])
yp, tl = Poincare(y0)

In [11]:
yp

array([ 18.8872526 ,  33.82895267,  45.27696802])

In [16]:
newton.newton_krylov_hook(P, yp[:2], r=0.0625)

DEBUG:yamath.newton:count:0	residue:5.953255e+01
DEBUG:yamath.newton:dx_norm:2.605245e+02
INFO:yamath.newton:hook step
DEBUG:yamath.newton:nu:0.000000e+00
DEBUG:yamath.newton:r:6.250000e-02
DEBUG:yamath.newton:Psi:5.889198e+04
DEBUG:yamath.newton:Psi:3.670845e-09
DEBUG:yamath.newton:Psi:1.468335e-08
DEBUG:yamath.newton:Psi:5.873296e-08
DEBUG:yamath.newton:Psi:2.349248e-07
DEBUG:yamath.newton:Psi:9.395862e-07
DEBUG:yamath.newton:Psi:3.756537e-06
DEBUG:yamath.newton:Psi:1.499729e-05
DEBUG:yamath.newton:Psi:5.953117e-05
DEBUG:yamath.newton:Psi:2.310293e-04
DEBUG:yamath.newton:Psi:8.237917e-04
DEBUG:yamath.newton:Psi:2.247334e-03
DEBUG:yamath.newton:Psi:3.622359e-03
DEBUG:yamath.newton:value:1.618212e+00
INFO:yamath.newton:Convergent into maximum
DEBUG:yamath.newton:count:1	residue:5.950546e+01
DEBUG:yamath.newton:dx_norm:2.553454e+02
INFO:yamath.newton:hook step
DEBUG:yamath.newton:nu:4.454577e+02
DEBUG:yamath.newton:r:6.250000e-02
DEBUG:yamath.newton:Psi:3.713256e-03
DEBUG:yamath.newton:

RuntimeError: Invalid trusted region